In [ ]:
# ISSUES TO TROUBLESHOOT

# 1: not returning all links -- only parent ones in this case:
wiki_scrape("Alba Flores")
# may be because there is a parent listed with no link so then skips the relatives

# 2: top_tv_links has duplicate links (len = 200 rather than 100) because there is a link in titleColumn and posterColumn

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import wikipediaapi as wiki

In [2]:
# Parent and Relative strings to search for in wikipedia infobox
parent_pattern = re.compile('Parent|Relative')

# url base
wiki_url_base = "https://en.wikipedia.org" # wikipedia
imdb_base_url = "https://www.imdb.com"

# wikipedia language setting
wiki_wiki = wiki.Wikipedia('en')

# imdb links
# Most Popular TV Shows as determined by IMDb Users
url_pop_tv = "https://www.imdb.com/chart/tvmeter?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=470df400-70d9-4f35-bb05-8646a1195842&pf_rd_r=DK1W37ZH61RXZP184X95&pf_rd_s=right-4&pf_rd_t=15506&pf_rd_i=toptv&ref_=chttvtp_ql_5"

# Most Popular Movies as determined by IMDb Users
url_pop_mov = "https://www.imdb.com/chart/moviemeter?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=470df400-70d9-4f35-bb05-8646a1195842&pf_rd_r=408FM1WEANBKDGP51PET&pf_rd_s=right-4&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_ql_2"

# Top Box Office (US) - updated weekly
url_box_office = "https://www.imdb.com/chart/boxoffice?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=470df400-70d9-4f35-bb05-8646a1195842&pf_rd_r=KK2KJN75YN0PGF8Q79QK&pf_rd_s=right-4&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_ql_1"

# IMDb Top 250 TV Shows - IMDb Top 250 as rated by regular IMDb voters
url_top_tv = "https://www.imdb.com/chart/toptv?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=470df400-70d9-4f35-bb05-8646a1195842&pf_rd_r=075QRGJ32Y0624A92Y2E&pf_rd_s=right-4&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_ql_6"

# IMDb Top 250 Movies - IMDb Top 250 as rated by regular IMDb voters
url_top_mov = "https://www.imdb.com/chart/top?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=470df400-70d9-4f35-bb05-8646a1195842&pf_rd_r=YSCMHSWYXMFCVH1D74MH&pf_rd_s=right-4&pf_rd_t=15506&pf_rd_i=tvmeter&ref_=chttvm_ql_3"


In [3]:
# given name of person, outputs parent links or false if none
def wiki_scrape(name):
    subject = name.replace(" ", "_")
    url = url_base + "/wiki/" + subject

    # check if wiki page exists
    wiki_page = wiki_wiki.page(subject)
    does_page_exist = wiki_page.exists()

    if does_page_exist is False :
        return [name, False] # no wiki page -> not famous enough, not a nepo baby
    
    elif does_page_exist is True : 
        data = requests.get(url).text
        soup = BeautifulSoup(data,'html.parser') # full page
        
        # check if infobox exists
        infobox = soup.find("table",{"class":"infobox biography vcard"}) # infobox
        
        if infobox is None :
            return [name, False] # no infobox on wiki page -> not a nepo baby
        else :
            does_parent_field_exist = bool(re.search("Parent|Relative", infobox.text)) # see if Parent or Relative field is listed in infobox
        
            if does_parent_field_exist is False :
                return [name, False]
            elif does_parent_field_exist is True :
                parent_field = soup.find('th', string=parent_pattern).parent
                parent_a_tags = parent_field.find_all('a')
                if len(parent_a_tags) == 0 :
                    return [name, False] # parents listed in infobox but not linked -> not a nepo baby
                else :
                    parent_wiki_list = []
                    for link in parent_field.find_all('a'):
                        parent_wiki = link.get('href')
                        parent_wiki_link = wiki_url_base + parent_wiki
                        parent_wiki_list.append(parent_wiki_link)
                        
                        parent_wiki_list[:] = [x for x in parent_wiki_list if "cite_note" not in x] # cited entries are in <a href> tags so remove those links here
    
                return [name, parent_wiki_list]


In [15]:
def get_imdb_list_links(url):
    page_data = requests.get(url).text
    soup = BeautifulSoup(page_data,'html.parser')
    
    list_of_media = soup.find("table", {"class":"chart full-width"}) 
    all_links = []
    
    for a in list_of_media.find_all('a'):
        link = a.get('href')
        link = imdb_base_url + link
        all_links.append(link)
    
   # df = pd.DataFrame(columns=["link"], data = all_links)

In [5]:
def top_cast(url):
    page_data =  requests.get(url).text
    soup = BeautifulSoup(page_data,'html.parser')
    
    title = soup.find("h1")
    title = title.text
    
    # grab top cast section of imdb page
    top_cast = soup.find_all("a", {"class": "sc-36c36dd0-1 QSQgP"})
    
    cast_list = []
    
    # get cast names
    for a in top_cast:
        cast_list.append(a.string)
        
    return[title, cast_list]

In [11]:
pop_tv_df = get_imdb_list_links(url_pop_tv)
pop_mov_df = get_imdb_list_links(url_pop_mov)
top_tv_df = get_imdb_list_links(url_top_tv)
top_mov_df = get_imdb_list_links(url_top_mov)
box_office_df = get_imdb_list_links(url_box_office)

In [7]:
pop_tv_df["imdb_info"] = pop_tv_df.apply(lambda row : top_cast(row['link']), axis=1)

AttributeError: 'NoneType' object has no attribute 'apply'

In [ ]:
# apply wiki_scrape function
#df["nepo"] = df.apply(lambda row : wiki_scrape(row['attendee']), axis=1)

In [ ]:
url = "https://en.wikipedia.org/wiki/Alba_Flores"

data = requests.get(url).text
soup = BeautifulSoup(data,'html.parser') # full page
        
# check if infobox exists
infobox = soup.find("table",{"class":"infobox biography vcard"}) # infobox

parent_field = soup.find('th', string=parent_pattern).parent
parent_a_tags = parent_field.find_all('a')

print(parent_field)

In [ ]:
wiki_scrape("Hailey Bieber")

In [10]:
print(pop_tv_df)

None


In [16]:
test = get_imdb_list_links("https://www.imdb.com/chart/toptv/")


In [17]:
type(test)

NoneType